# Arguments

In [1]:
batch_size = 32
dataset_path_plantvillage = '../datasets/plantvillage'
dataset_path_self = '../datasets/self'
dataset_path_validation = '../datasets/validation'

dataset_paths = [dataset_path_plantvillage, dataset_path_self, dataset_path_validation]

warmup_model_path = 'model-warmup.h5'
model_save_path = 'model-final.h5'
checkpoint_path = 'checkpoints-finetuning.hdf5'

input_width = 224
input_height = 224
input_depth = 3

# --------------------------------------------------
num_of_epochs = 100
start_epoch = 0
# --------------------------------------------------


# Select training classes

In [2]:
class_paths_training = ['../datasets/self/___Early_blight', '../datasets/self/___Appids', '../datasets/self/___Leaf_miner', '../datasets/self/___Curly_top_virus']

# Load dataset

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image   import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.layers import Input
from keras.models import Model
from keras .applications import VGG16
from keras.optimizers import SGD
from keras.models import Model
from imutils import paths
import numpy as np
import os

import sys
sys.path.append('..')

from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader

# construct the image generator for data augmentation
aug = ImageDataGenerator(   rotation_range=30,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

def load_datasets(path_list):
    # Load image paths
    image_paths = []
    print("[INFO] loading images...")
    for path in path_list:
        image_paths.extend(list(paths.list_images(path)))
        
    # Get unique classnames

    class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
    class_names = [str(x) for x in np.unique(class_names)]

    # Initial image preprocessing
    aap = AspectAwarePreprocessor(input_width, input_height)
    iap= ImageToArrayPreprocessor()

    #Load image data and perform image data preprocessing
    sdl = SimpleDatasetLoader(preprocessors=[aap,iap])
    (data,labels)  = sdl.load(image_paths,verbose=500)
    data = data.astype("float") / 255.0


    # train test split
    (train_x,test_x,train_y,test_y) = train_test_split(data,labels,test_size=0.25,random_state=42)

    # convert the labels from integers to vectors
    train_y = LabelBinarizer().fit_transform(train_y)
    test_y = LabelBinarizer().fit_transform(test_y)
    
    return (train_x,test_x,train_y,test_y, class_names)

(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_training)



Using TensorFlow backend.


[INFO] loading images...
[INFO]: Processed 500/4849
[INFO]: Processed 1000/4849
[INFO]: Processed 1500/4849
[INFO]: Processed 2000/4849
[INFO]: Processed 2500/4849
[INFO]: Processed 3000/4849
[INFO]: Processed 3500/4849
[INFO]: Processed 4000/4849
[INFO]: Processed 4500/4849


# Load model from disk

In [4]:
# import tensorflow as tf
# print(tf.__version__)


# model = tf.keras.models.load_model(
#     warmup_model_path,
#     custom_objects=None,
#     compile=False
# )

from keras.models import load_model

model = load_model(warmup_model_path)

# Print index of layers

In [5]:
# iterate for all layers in the network and print its' index value
for (i,layer) in enumerate(model.layers):
    print("[INFO] {:5}\t{:30}{}".format(i, layer.name, layer.__class__.__name__))

[INFO]     0	input_1                       InputLayer
[INFO]     1	block1_conv1                  Conv2D
[INFO]     2	block1_conv2                  Conv2D
[INFO]     3	block1_pool                   MaxPooling2D
[INFO]     4	block2_conv1                  Conv2D
[INFO]     5	block2_conv2                  Conv2D
[INFO]     6	block2_pool                   MaxPooling2D
[INFO]     7	block3_conv1                  Conv2D
[INFO]     8	block3_conv2                  Conv2D
[INFO]     9	block3_conv3                  Conv2D
[INFO]    10	block3_pool                   MaxPooling2D
[INFO]    11	block4_conv1                  Conv2D
[INFO]    12	block4_conv2                  Conv2D
[INFO]    13	block4_conv3                  Conv2D
[INFO]    14	block4_pool                   MaxPooling2D
[INFO]    15	block5_conv1                  Conv2D
[INFO]    16	block5_conv2                  Conv2D
[INFO]    17	dropout_2                     Dropout
[INFO]    18	block5_conv3                  Conv2D
[INFO]    19	dropout_

# Unfreeze final CONV layers

In [6]:
for layer in model.layers[:15]:
   layer.trainable = False
for layer in model.layers[15:]:
   layer.trainable = True
print('unfrozen')

unfrozen


# Compile model

In [7]:
from keras.optimizers import SGD

# Build the model from the new
print("[INFO] re-compiling model ...")
opt = SGD(lr=0.0001, momentum=0.09)
# Fine-tuning with a small learning rate
model.compile(loss = 'categorical_crossentropy',optimizer = opt,
              metrics=['accuracy'])


[INFO] re-compiling model ...


# Checkpoints

In [8]:
from keras.callbacks import ModelCheckpoint


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', mode='min', 
save_best_only=True, verbose=1)

callbacks = [checkpoint]



In [9]:
#load checkpoints if existing

import os

if(os.path.exists(checkpoint_path)):
    model.load_weights(checkpoint_path)

# Training Monitor

In [10]:
from utils.callbacks.training_monitor import TrainingMonitor
import pathlib
import json
import os

fig_path = "plot"
json_path = "values.json"
values_path = 'values.json'

callbacks.append(TrainingMonitor(fig_path, json_path, start_epoch))

# Fit model

In [ ]:
num_of_epochs = 100

H = model.fit_generator(
    aug.flow(train_x,train_y, batch_size = 32),
             validation_data = (test_x,test_y),
             epochs=num_of_epochs,
             steps_per_epoch = len(train_x) //32,
             verbose = 1,
             callbacks=callbacks)

Epoch 1/100
113/113 [==============================] - 67s 589ms/step - loss: 0.4192 - accuracy: 0.8405 - val_loss: 0.3125 - val_accuracy: 0.9077

Epoch 00001: val_loss improved from inf to 0.31248, saving model to checkpoints-finetuning.hdf5
Epoch 2/100
113/113 [==============================] - 59s 519ms/step - loss: 0.3918 - accuracy: 0.8532 - val_loss: 0.2919 - val_accuracy: 0.9184

Epoch 00002: val_loss improved from 0.31248 to 0.29187, saving model to checkpoints-finetuning.hdf5
Epoch 3/100
113/113 [==============================] - 58s 511ms/step - loss: 0.3765 - accuracy: 0.8554 - val_loss: 0.3217 - val_accuracy: 0.9068

Epoch 00003: val_loss did not improve from 0.29187
Epoch 4/100
113/113 [==============================] - 57s 509ms/step - loss: 0.3400 - accuracy: 0.8638 - val_loss: 0.2790 - val_accuracy: 0.9217

Epoch 00004: val_loss improved from 0.29187 to 0.27901, saving model to checkpoints-finetuning.hdf5
Epoch 5/100
113/113 [==============================] - 57s 502ms/

# Save model

In [ ]:
model.save(model_save_path)

# Evaluate with 20%

In [ ]:
from sklearn.metrics import classification_report

print("[INFO] evaluating after initialization...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))

# Evaluate with validation images

In [ ]:
class_paths_validation = ['../datasets/validation/___Early_blight', '../datasets/validation/___Appids', '../datasets/validation/___Leaf_miner', '../datasets/self/___Curly_top_virus']

In [ ]:
(train_x,test_x,train_y,test_y, class_names) = load_datasets(class_paths_validation)

from sklearn.metrics import classification_report

print("[INFO] evaluating with validation set...")
predictions = model.predict(test_x,batch_size=batch_size)

print(classification_report(test_y.argmax(axis =1),
                            predictions.argmax(axis =1),
                            target_names=class_names, 
                            digits=4))